# Aqsis Examples #

This notebook tries rendering some of the example scenes that come with [Aqsis](http://www.aqsis.org/), the Free Software implementation of Pixar’s RenderMan language. It assumes the relevant files are present in the usual place on a Debian installation; you may need to edit the relevant paths for some other OS.

Besides Aqsis, you will need my my [ipy_magics](https://github.com/ldo/ipy_magics), specifically `rman_magic.py`, which provides an interface to Aqsis from a Jupyter notebook.

In [ ]:
# Edit this as appropriate for the correct path to the Python module
%run ../ipy_magics/rman_magic.py

## Your First Steps With Aqsis ## 

The following is adapted from “[Your First Steps with Aqsis](http://www.aqsis.org/documentation/tutorials/first_steps.html)”.

First, the sphere is rendered with a constant colour:

In [ ]:
%%rman
%rib
##RenderMan RIB-Structure 1.0

%autodisplay

FrameBegin 1

Format 640 480 -1
ShadingRate 1
Projection "perspective" "fov" [30]
FrameAspectRatio 1.33
Identity
 
# Default distant headlight
LightSource "distantlight" 1
 
# Camera transformation
Translate 0 0 5
 
WorldBegin

Identity

AttributeBegin
 
Color [1.0 0.6 0.0] # A nice orange color
Surface "plastic" "Ka" [1] "Kd" [0.5] "Ks" 1 "roughness" 0.1
TransformBegin
  Rotate 90 1 0 0   # Make +z point "up". The default camera coordinate system has
  Sphere 1 -1 1 360 # +y up, but the Sphere primitive has its poles on the z axis.
TransformEnd

AttributeEnd

WorldEnd
 
FrameEnd

Next, the material is changed to a custom shader using a blotchy Perlin noise, which is emissive (does not respond to lighting):

In [ ]:
%%rman
%sl simple_surface
// A very simple SL surface shader
surface simple_surface() {
  Ci = noise(10*P);
}
%rib
##RenderMan RIB-Structure 1.0

%autodisplay

FrameBegin 1

Format 640 480 -1
ShadingRate 1
Projection "perspective" "fov" [30]
FrameAspectRatio 1.33
Identity
 
# Default distant headlight
LightSource "distantlight" 1
 
# Camera transformation
Translate 0 0 5
 
WorldBegin

Identity

AttributeBegin
 
Color [1.0 0.6 0.0] # A nice orange color
Surface "simple_surface"
TransformBegin
  Rotate 90 1 0 0   # Make +z point "up". The default camera coordinate system has
  Sphere 1 -1 1 360 # +y up, but the Sphere primitive has its poles on the z axis.
TransformEnd

AttributeEnd

WorldEnd
 
FrameEnd

Change the material shader definition so it responds to lighting in a diffuse way:

In [ ]:
%%rman
%sl simple_surface
// A very simple SL surface shader
surface simple_surface() {
  color Cd = noise(10*P);
  Ci = Cd * diffuse(N);
}
%rib
##RenderMan RIB-Structure 1.0

%autodisplay

FrameBegin 1

Format 640 480 -1
ShadingRate 1
Projection "perspective" "fov" [30]
FrameAspectRatio 1.33
Identity
 
# Default distant headlight
LightSource "distantlight" 1
 
# Camera transformation
Translate 0 0 5
 
WorldBegin

Identity

AttributeBegin
 
Color [1.0 0.6 0.0] # A nice orange color
Surface "simple_surface"
TransformBegin
  Rotate 90 1 0 0   # Make +z point "up". The default camera coordinate system has
  Sphere 1 -1 1 360 # +y up, but the Sphere primitive has its poles on the z axis.
TransformEnd

AttributeEnd

WorldEnd
 
FrameEnd

Now revert to the original shiny orange colour, and try applying a displacement shader to make the surface look bumpy:

In [ ]:
%%rman
%sl simple_displacement
// A very simple SL displacement shader
displacement simple_displacement() {
  point Ptex = P + noise(P*2);
  P = P + N * 0.05 * float noise(20*Ptex);
  N = calculatenormal(P);  
}
%rib
##RenderMan RIB-Structure 1.0

%autodisplay

FrameBegin 1

Format 640 480 -1
ShadingRate 1
Projection "perspective" "fov" [30]
FrameAspectRatio 1.33
Identity
 
# Default distant headlight
LightSource "distantlight" 1
 
# Camera transformation
Translate 0 0 5
 
WorldBegin

Identity

AttributeBegin
 
Color [1.0 0.6 0.0] # A nice orange color
# Displacement bounds, important for correct rendering
Attribute "displacementbound" "float sphere" [0.1] "coordinatesystem" ["shader"]
Surface "plastic" "Ka" [1] "Kd" [0.5] "Ks" 1 "roughness" 0.1
Displacement "simple_displacement"
TransformBegin
  Rotate 90 1 0 0   # Make +z point "up". The default camera coordinate system has
  Sphere 1 -1 1 360 # +y up, but the Sphere primitive has its poles on the z axis.
TransformEnd

AttributeEnd

WorldEnd
 
FrameEnd

## Vase Example #

In [ ]:
%%rman --source-search="&:/usr/share/aqsis/examples/scenes/vase:/usr/share/aqsis/shaders"
%slfile displacement/dented.sl
%slfile light/shadowspot.sl
%ribfile vase.rib


## Fisheye Example ##

In [ ]:
%%rman --resource-search="/usr/share/aqsis/examples/scenes/fisheye:&" --verbose=0
%slfile fisheye_projection.sl
%ribfile envmap.rib
%ribfile fisheye.rib
%ribfile scene.rib


## Bicycle Example ##

In [ ]:
%%rman --resource-search=/usr/share/aqsis/examples/features/archives
%ribfile --autodisplay bike.rib

## Constructive Solid Geometry Example ##

In [ ]:
%%rman
%ribfile /usr/share/aqsis/examples/features/solidmodeling/csg.rib

## The (In)Famous Cornell Box ##

Note that this render can be quite slow, taking nearly a minute on my machine.

In [ ]:
%%rman --timed --resource-search="&:/usr/share/aqsis/examples/point_based_gi/cornellbox" --archive-search="&:/usr/share/aqsis/examples/point_based_gi/cornellbox"
%slfile ao.sl
%slfile bake_points.sl
%slfile indirect.sl
%ribfile all_passes.rib